In [47]:
import pandas as pd
import numpy as np
import yfinance as yf
from google.colab import files
import time

#assuming that all the companies are part of nifty 50 from 2020 to 2023
stocks = ['RELIANCE.NS', 'TCS.NS', 'HDFCBANK.NS', 'INFY.NS', 'HINDUNILVR.NS',
    'ICICIBANK.NS', 'KOTAKBANK.NS', 'SBIN.NS', 'BHARTIARTL.NS', 'BAJFINANCE.NS',
    'ASIANPAINT.NS', 'HCLTECH.NS', 'AXISBANK.NS', 'LT.NS', 'ITC.NS',
    'SUNPHARMA.NS', 'ULTRACEMCO.NS', 'NESTLEIND.NS', 'WIPRO.NS', 'POWERGRID.NS',
    'TECHM.NS', 'ADANIPORTS.NS', 'TITAN.NS', 'COALINDIA.NS', 'BPCL.NS',
    'DIVISLAB.NS', 'DRREDDY.NS', 'EICHERMOT.NS', 'GRASIM.NS', 'HDFCLIFE.NS',
    'HEROMOTOCO.NS', 'HINDALCO.NS', 'JSWSTEEL.NS', 'M&M.NS', 'MARUTI.NS',
    'NTPC.NS', 'ONGC.NS', 'SBILIFE.NS', 'TATASTEEL.NS', 'TATAMOTORS.NS',
    'TATACONSUM.NS', 'INDUSINDBK.NS', 'BAJAJ-AUTO.NS', 'BRITANNIA.NS', 'CIPLA.NS',
    'ADANIENT.NS', 'APOLLOHOSP.NS', 'BAJAJFINSV.NS', 'UPL.NS', 'LTIM.NS'

]

#historical price data
data = yf.download(stocks, start="2020-01-01", end="2023-12-31", interval="1mo")['Adj Close']

[*********************100%%**********************]  50 of 50 completed


In [48]:
monthly_returns = data.pct_change().dropna()
formation_period = 12
holding_period = 1
trades = pd.DataFrame(columns=['Year', 'Month', 'Stock', 'Position', 'Buy Price', 'Sell Price', 'Profit', 'Drawdown', 'Upside', 'Monthly Return', 'Cumulative Return', 'Mean Monthly Return', 'Monthly Volatility', 'Risk-Adjusted Mean Return'])
portfolio_returns = pd.DataFrame(columns=['Year', 'Month', 'Portfolio Type', 'Portfolio Return'] + stocks)

In [49]:
#strategy
for end in range(formation_period, len(monthly_returns)):
    start = end - formation_period
    formation_data = monthly_returns.iloc[start:end]

    #metrics
    cumulative_return = (formation_data + 1).prod() - 1
    mean_monthly_return = formation_data.mean()
    monthly_volatility = formation_data.std()
    risk_adjusted_mean_return = mean_monthly_return / monthly_volatility

    #sort stocks based on cumulative return and select top & bottom decile
    top_decile_stocks = cumulative_return.nlargest(int(len(cumulative_return) / 10)).index
    bottom_decile_stocks = cumulative_return.nsmallest(int(len(cumulative_return) / 10)).index

    #trades for longonly portfolio
    for stock in top_decile_stocks:
        buy_price = data.iloc[end][stock]
        sell_price = data.iloc[end + holding_period][stock] if end + holding_period < len(data) else np.nan
        profit = (sell_price - buy_price) / buy_price if not np.isnan(sell_price) else np.nan
        drawdown = formation_data[stock].min()
        upside = formation_data[stock].max()

        trade = {
            'Year': data.index[end].year,
            'Month': data.index[end].month,
            'Stock': stock,
            'Position': 'Buy',
            'Buy Price': buy_price,
            'Sell Price': sell_price,
            'Profit': profit,
            'Drawdown': drawdown,
            'Upside': upside,
            'Monthly Return': mean_monthly_return[stock],
            'Cumulative Return': cumulative_return[stock],
            'Mean Monthly Return': mean_monthly_return[stock],
            'Monthly Volatility': monthly_volatility[stock],
            'Risk-Adjusted Mean Return': risk_adjusted_mean_return[stock]
        }
        trades = pd.concat([trades, pd.DataFrame([trade])], ignore_index=True)

    #portfolio return and weights for longonly portfolio
    if end + holding_period < len(data):
        selected_data = data[top_decile_stocks]
        selected_data_end = selected_data.iloc[end]
        selected_data_next = selected_data.iloc[end + holding_period]

        portfolio_return = selected_data_next.mean() / selected_data_end.mean() - 1
        weights = selected_data_end / selected_data_end.sum()

        portfolio_entry = {'Year': data.index[end].year, 'Month': data.index[end].month, 'Portfolio Type': 'Long-Only', 'Portfolio Return': portfolio_return}
        portfolio_entry.update(weights.to_dict())

        portfolio_returns = pd.concat([portfolio_returns, pd.DataFrame([portfolio_entry])], ignore_index=True)

    #trades for shortonly portfolio
    for stock in bottom_decile_stocks:
        sell_price = data.iloc[end][stock]
        buy_price = data.iloc[end + holding_period][stock] if end + holding_period < len(data) else np.nan
        profit = (sell_price - buy_price) / sell_price if not np.isnan(buy_price) else np.nan
        drawdown = formation_data[stock].min()
        upside = formation_data[stock].max()

        trade = {
            'Year': data.index[end].year,
            'Month': data.index[end].month,
            'Stock': stock,
            'Position': 'Short',
            'Buy Price': buy_price,
            'Sell Price': sell_price,
            'Profit': profit,
            'Drawdown': drawdown,
            'Upside': upside,
            'Monthly Return': mean_monthly_return[stock],
            'Cumulative Return': cumulative_return[stock],
            'Mean Monthly Return': mean_monthly_return[stock],
            'Monthly Volatility': monthly_volatility[stock],
            'Risk-Adjusted Mean Return': risk_adjusted_mean_return[stock]
        }
        trades = pd.concat([trades, pd.DataFrame([trade])], ignore_index=True)

    #portfolio return and weights for shortonly portfolio
    if end + holding_period < len(data):
        selected_data = data[bottom_decile_stocks]
        selected_data_end = selected_data.iloc[end]
        selected_data_next = selected_data.iloc[end + holding_period]

        portfolio_return = selected_data_end.mean() / selected_data_next.mean() - 1
        weights = -selected_data_end / selected_data_end.sum()

        portfolio_entry = {'Year': data.index[end].year, 'Month': data.index[end].month, 'Portfolio Type': 'Short-Only', 'Portfolio Return': portfolio_return}
        portfolio_entry.update(weights.to_dict())

        portfolio_returns = pd.concat([portfolio_returns, pd.DataFrame([portfolio_entry])], ignore_index=True)


In [50]:
trades

,Year,Month,Stock,Position,Buy Price,Sell Price,Profit,Drawdown,Upside,Monthly Return,Cumulative Return,Mean Monthly Return,Monthly Volatility,Risk-Adjusted Mean Return
0,2021,1,ADANIENT.NS,Buy,505.389221,831.575500,0.645416,-0.370684,0.595150,0.089642,1.224259,0.089642,0.216944,0.413201
1,2021,1,LTIM.NS,Buy,3779.448486,3432.338867,-0.091841,-0.251970,0.234384,0.069596,1.068867,0.069596,0.120503,0.577548
2,2021,1,CIPLA.NS,Buy,804.167786,766.340027,-0.047040,-0.100246,0.397969,0.058906,0.852865,0.058906,0.125510,0.469337
3,2021,1,WIPRO.NS,Buy,410.957062,404.390442,-0.015979,-0.111763,0.279080,0.053694,0.771905,0.053694,0.106756,0.502961
4,2021,1,DIVISLAB.NS,Buy,3277.400635,3270.835938,-0.002003,-0.122869,0.193480,0.051658,0.738971,0.051658,0.100607,0.513468
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
345,2023,11,ADANIENT.NS,Short,2847.801514,2357.599365,-0.207924,-0.541393,0.295281,-0.013173,-0.397404,-0.013173,0.219294,-0.060070
346,2023,11,UPL.NS,Short,587.250000,570.650024,-0.029090,-0.122921,0.059573,-0.023112,-0.265521,-0.023112,0.069539,-0.332364
347,2023,11,KOTAKBANK.NS,Short,1908.099976,1755.449951,-0.086958,-0.083302,0.118418,-0.007237,-0.097663,-0.007237,0.053508,-0.135252
348,2023,11,INFY.NS,Short,1534.253052,1446.994751,-0.060303,-0.122693,0.077159,-0.005837,-0.086335,-0.005837,0.059380,-0.098306


In [51]:
years = pd.RangeIndex(start=portfolio_returns['Year'].min(), stop=portfolio_returns['Year'].max() + 1, step=1)
months = pd.RangeIndex(start=1, stop=13, step=1)

complete_date_range = pd.MultiIndex.from_product([years, months], names=['Year', 'Month'])
complete_df = pd.DataFrame(index=complete_date_range).reset_index()
overall_portfolio = portfolio_returns.pivot_table(index=['Year', 'Month'], values=['Portfolio Return'] + stocks, fill_value=0)
overall_portfolio = overall_portfolio.fillna(0).reset_index()
overall_portfolio = pd.merge(complete_df, overall_portfolio, on=['Year', 'Month'], how='left').fillna(0)
overall_portfolio = overall_portfolio[['Year', 'Month', 'Portfolio Return'] + stocks]
overall_portfolio.to_csv('overall_portfolio.csv', index=False)
overall_portfolio

,Year,Month,Portfolio Return,RELIANCE.NS,TCS.NS,HDFCBANK.NS,INFY.NS,HINDUNILVR.NS,ICICIBANK.NS,KOTAKBANK.NS,...,TATACONSUM.NS,INDUSINDBK.NS,BAJAJ-AUTO.NS,BRITANNIA.NS,CIPLA.NS,ADANIENT.NS,APOLLOHOSP.NS,BAJAJFINSV.NS,UPL.NS,LTIM.NS
0,2021,1,-0.064876,0.000000,0.00000,0.0,0.000000,0.000000,0.0,0.000000,...,0.0,-0.427945,0.000000,0.000000,0.091618,0.057579,0.000000,0.000000,0.000000,0.430590
1,2021,2,0.016332,0.000000,0.00000,0.0,0.000000,-0.327173,0.0,0.000000,...,0.0,-0.166252,0.000000,0.000000,0.263271,0.285682,0.000000,0.000000,0.000000,0.000000
2,2021,3,0.105541,0.000000,0.00000,0.0,0.000000,-0.365669,0.0,-0.277710,...,0.0,0.000000,0.000000,0.000000,0.000000,0.483759,0.000000,0.000000,0.000000,0.000000
3,2021,4,0.029040,0.000000,0.00000,0.0,0.000000,-0.290160,0.0,0.000000,...,0.0,0.000000,0.000000,-0.425063,0.000000,0.457175,0.000000,0.000000,0.000000,0.000000
4,2021,5,0.016800,0.000000,0.00000,0.0,0.000000,-0.281455,0.0,0.000000,...,0.0,0.000000,0.000000,-0.421447,0.000000,0.166248,0.000000,0.000000,0.000000,0.000000
5,2021,6,0.045865,0.000000,0.00000,0.0,0.000000,0.000000,0.0,0.000000,...,0.0,0.000000,0.000000,-0.410202,0.000000,0.245515,0.585542,0.000000,0.000000,0.000000
6,2021,7,-0.006505,-0.145032,0.00000,0.0,0.000000,0.000000,0.0,0.000000,...,0.0,0.000000,0.000000,-0.257173,0.000000,0.487891,0.000000,0.000000,0.000000,0.000000
7,2021,8,-0.054930,-0.104678,0.00000,0.0,0.000000,0.000000,0.0,0.000000,...,0.0,0.000000,0.000000,-0.195356,0.000000,0.177241,0.550697,0.191509,0.000000,0.000000
8,2021,9,0.021951,-0.111288,0.00000,0.0,0.000000,0.000000,0.0,0.000000,...,0.0,0.000000,0.000000,-0.183904,0.000000,0.356771,0.000000,0.432379,0.000000,0.000000
9,2021,10,0.029730,0.000000,0.00000,0.0,0.000000,0.000000,0.0,0.000000,...,0.0,0.000000,0.000000,-0.179627,0.000000,0.339196,0.000000,0.424750,0.000000,0.000000


In [52]:
#save files
from google.colab import drive
drive.mount('/content/drive')


trade_folder = '/content/drive/My Drive/AStratInvest/Stock_Trades/'
drive_folder = '/content/drive/My Drive/AStratInvest/'


import os
if not os.path.exists(trade_folder):
    os.makedirs(trade_folder)
#individual stock trade
for stock in stocks:
    stock_trades = trades[trades['Stock'] == stock]
    output_file = os.path.join(trade_folder, f'{stock}_Trades.csv')
    stock_trades.to_csv(output_file, index=False)
    print(f'Saved file to {output_file}')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Saved file to /content/drive/My Drive/AStratInvest/Stock_Trades/RELIANCE.NS_Trades.csv
Saved file to /content/drive/My Drive/AStratInvest/Stock_Trades/TCS.NS_Trades.csv
Saved file to /content/drive/My Drive/AStratInvest/Stock_Trades/HDFCBANK.NS_Trades.csv
Saved file to /content/drive/My Drive/AStratInvest/Stock_Trades/INFY.NS_Trades.csv
Saved file to /content/drive/My Drive/AStratInvest/Stock_Trades/HINDUNILVR.NS_Trades.csv
Saved file to /content/drive/My Drive/AStratInvest/Stock_Trades/ICICIBANK.NS_Trades.csv
Saved file to /content/drive/My Drive/AStratInvest/Stock_Trades/KOTAKBANK.NS_Trades.csv
Saved file to /content/drive/My Drive/AStratInvest/Stock_Trades/SBIN.NS_Trades.csv
Saved file to /content/drive/My Drive/AStratInvest/Stock_Trades/BHARTIARTL.NS_Trades.csv
Saved file to /content/drive/My Drive/AStratInvest/Stock_Trades/BAJFINANCE.NS_Trades.csv
Saved 

In [53]:
#overall portfolio
overall_portfolio_file_path = os.path.join(drive_folder, 'overall_portfolio.csv')
overall_portfolio.to_csv(overall_portfolio_file_path, index=False)
print(f'Saved file to {overall_portfolio_file_path}')

Saved file to /content/drive/My Drive/AStratInvest/overall_portfolio.csv


In [54]:
#trade sheet
trades_sheet_path = os.path.join(drive_folder, 'main_trades_sheet.csv')
trades.to_csv(trades_sheet_path, index=False)
print(f'Saved file to {trades_sheet_path}')

Saved file to /content/drive/My Drive/AStratInvest/main_trades_sheet.csv
